**Connect GoogleDrive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Unzip file dataset**

In [ ]:
!unzip "/content/drive/MyDrive/flowers.zip" -d "/content/drive/MyDrive/flowers/"

# **VGG16**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
from PIL import Image
import os
import logging
from datetime import datetime

# CẤU HÌNH ĐƯỜNG DẪN VÀ THAM SỐ
path_root='flower102'
model_name='vgg16'
CONFIG = {
    'data_dir': f'/content/drive/MyDrive/flowers/{path_root}',  # Thư mục chứa dữ liệu
    'model_path': f'/content/drive/MyDrive/model/{path_root}/vgg16_flower_model.pth',  # Đường dẫn lưu/tải mô hình
    'image_path': f'/content/drive/MyDrive/flowers/{path_root}/val/english_marigold/image_05188.jpg',  # Đường dẫn ảnh dự đoán mẫu
    'log_dir': f'/content/drive/MyDrive/logs/{path_root}/{model_name}',  # Thư mục lưu log
    'batch_size': 32,  # Kích thước batch
    'num_epochs': 10,  # Số epoch huấn luyện
    'learning_rate': 0.001,  # Tốc độ học
    'momentum': 0.9,  # Momentum cho SGD
    'num_workers': 2,  # Số worker cho DataLoader
}


# Thiết lập logging
if not os.path.exists(CONFIG['log_dir']):
    os.makedirs(CONFIG['log_dir'])
log_file = os.path.join(CONFIG['log_dir'], f'train_log_vgg16_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

# Cấu hình logger với file handler
logger = logging.getLogger()
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Ghi log ban đầu
logger.info("Bắt đầu chương trình với VGG16")
logger.info(f"Device: {torch.cuda.is_available() and 'cuda' or 'cpu'}")
logger.info(f"Cấu hình: {CONFIG}")

# Thiết lập device (GPU T4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# 1. Chuẩn bị dữ liệu
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_dataset = ImageFolder(root=f'{CONFIG["data_dir"]}/train', transform=data_transforms['train'])
val_dataset = ImageFolder(root=f'{CONFIG["data_dir"]}/test', transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=CONFIG['num_workers'])
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=CONFIG['num_workers'])

# Lấy danh sách tên loài hoa
class_names = train_dataset.classes
logger.info(f"Số loài hoa: {len(class_names)}")
print(f"Số loài hoa: {len(class_names)}")

# 2. Tải hoặc khởi tạo mô hình VGG16
model = vgg16(pretrained=True)
num_classes = len(class_names)
# Thay đổi tầng phân loại (classifier) cuối cùng của VGG16
model.classifier[6] = nn.Linear(4096, num_classes)  # VGG16 có 4096 đặc trưng ở tầng trước cuối
model = model.to(device)

# Kiểm tra xem mô hình đã tồn tại chưa
if os.path.exists(CONFIG['model_path']):
    logger.info(f"Tải mô hình đã lưu từ {CONFIG['model_path']}")
    print(f"Tải mô hình đã lưu từ {CONFIG['model_path']}...")
    model.load_state_dict(torch.load(CONFIG['model_path']))
else:
    # 3. Định nghĩa hàm mất mát và optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=CONFIG['learning_rate'], momentum=CONFIG['momentum'])

    # 4. Huấn luyện mô hình
    def train_model(model, train_loader, criterion, optimizer, num_epochs=CONFIG['num_epochs']):
        model.train()
        logger.info("Đang huấn luyện mô hình VGG16...")
        print("Đang huấn luyện mô hình VGG16...")
        for epoch in range(num_epochs):
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
            log_msg = f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}'
            logger.info(log_msg)
            print(log_msg)

    train_model(model, train_loader, criterion, optimizer)

    # 5. Lưu mô hình
    torch.save(model.state_dict(), CONFIG['model_path'])
    logger.info(f"Mô hình đã được lưu vào {CONFIG['model_path']}")
    print(f"Mô hình đã được lưu vào {CONFIG['model_path']}")

# 6. Đánh giá và dự đoán
def evaluate_and_predict(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    logger.info("Đánh giá mô hình VGG16...")
    print("Đánh giá mô hình VGG16...")
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Dự đoán một ảnh từ batch đầu tiên
            probs = torch.nn.functional.softmax(outputs[0], dim=0)
            confidence = torch.max(probs).item() * 100
            flower_name = class_names[predicted[0]]
            log_msg = f"Dự đoán: {flower_name}, Độ chính xác: {confidence:.2f}%"
            logger.info(log_msg)
            print(log_msg)
            break  # Chỉ in một ví dụ

    accuracy = 100 * correct / total
    log_msg = f"Độ chính xác trên tập kiểm tra: {accuracy:.2f}%"
    logger.info(log_msg)
    print(log_msg)

evaluate_and_predict(model, val_loader)

# 7. Dự đoán một ảnh cụ thể
def predict_single_image(image_path, model, transform, class_names):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        probs = torch.nn.functional.softmax(outputs[0], dim=0)
        confidence = torch.max(probs).item() * 100
        predicted_idx = torch.argmax(probs).item()
        flower_name = class_names[predicted_idx]
        log_msg = f"Dự đoán ảnh: {flower_name}, Độ chính xác: {confidence:.2f}%"
        logger.info(log_msg)
        print(f"Dự đoán ảnh: {flower_name}, Độ chính xác: {confidence:.2f}%")

logger.info("Dự đoán một ảnh cụ thể với VGG16:")
print("Dự đoán một ảnh cụ thể với VGG16:")
predict_single_image(CONFIG['image_path'], model, data_transforms['val'], class_names)

logger.info("Kết thúc chương trình")
print("Kết thúc chương trình")

# Đảm bảo log được lưu
file_handler.close()

INFO:root:Bắt đầu chương trình với VGG16
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/ba

Device: cuda
Số loài hoa: 102


INFO:root:Tải mô hình đã lưu từ /content/drive/MyDrive/model/flower102/vgg16_flower_model.pth
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop

Tải mô hình đã lưu từ /content/drive/MyDrive/model/flower102/vgg16_flower_model.pth...


INFO:root:Đánh giá mô hình VGG16...


Đánh giá mô hình VGG16...


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

Dự đoán: alpine_sea_holly, Độ chính xác: 99.83%
Độ chính xác trên tập kiểm tra: 96.88%
Dự đoán một ảnh cụ thể với VGG16:
Dự đoán ảnh: english_marigold, Độ chính xác: 65.38%
Kết thúc chương trình


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

# **VGG16 + KNN K=3,5,7**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16, alexnet
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
import os
import logging
from datetime import datetime
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pickle


# CẤU HÌNH ĐƯỜNG DẪN VÀ THAM SỐ
path_root = 'flower17'
CONFIG = {
    'data_dir': f'/content/drive/MyDrive/flowers/{path_root}',
    'model_vgg16_path': f'/content/drive/MyDrive/model/{path_root}/vgg16_flower_model.pth',
    'model_alexnet_path': f'/content/drive/MyDrive/model/{path_root}/alexnet_flower_model.pth',
    'model_knn_path': '/content/drive/MyDrive/model/{path_root}/knn_flower_model_k{k}.pkl',  # Sẽ định dạng sau
    'image_path': f'/content/drive/MyDrive/flowers/{path_root}/val/tulip/image_0567.jpg',
    'log_dir': '/content/drive/MyDrive/logs/{path_root}/knn{k}',  # Sẽ định dạng sau
    'batch_size': 32,
    'num_epochs': 10,
    'learning_rate': 0.001,
    'momentum': 0.9,
    'num_workers': 2,
    'k': 3  # Tham số K cho KNN
}

# Định dạng đường dẫn với k
CONFIG['model_knn_path'] = CONFIG['model_knn_path'].format(path_root=path_root, k=CONFIG['k'])
CONFIG['log_dir'] = CONFIG['log_dir'].format(path_root=path_root, k=CONFIG['k'])

# Thiết lập logging
if not os.path.exists(CONFIG['log_dir']):
    os.makedirs(CONFIG['log_dir'], exist_ok=True)
log_file = os.path.join(CONFIG['log_dir'], f'train_log_knn{CONFIG["k"]}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

logger = logging.getLogger()
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Ghi log ban đầu
logger.info("Bắt đầu chương trình với Ensemble VGG16, AlexNet và KNN")
logger.info(f"Device: {torch.cuda.is_available() and 'cuda' or 'cpu'}")
logger.info(f"Cấu hình: {CONFIG}")

# Thiết lập device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# 1. Chuẩn bị dữ liệu
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_dataset = ImageFolder(root=f'{CONFIG["data_dir"]}/train', transform=data_transforms['train'])
val_dataset = ImageFolder(root=f'{CONFIG["data_dir"]}/val', transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=CONFIG['num_workers'])
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=CONFIG['num_workers'])

# Lấy danh sách tên loài hoa
class_names = train_dataset.classes
logger.info(f"Số loài hoa: {len(class_names)}")
print(f"Số loài hoa: {len(class_names)}")

# 2. Tải mô hình VGG16
model_vgg16 = vgg16(pretrained=False)
num_classes = len(class_names)
model_vgg16.classifier[6] = nn.Linear(4096, num_classes)
if os.path.exists(CONFIG['model_vgg16_path']):
    logger.info(f"Tải mô hình VGG16 từ {CONFIG['model_vgg16_path']}")
    print(f"Tải mô hình VGG16 từ {CONFIG['model_vgg16_path']}...")
    model_vgg16.load_state_dict(torch.load(CONFIG['model_vgg16_path'], map_location=device))
else:
    logger.error(f"Không tìm thấy file mô hình VGG16 tại {CONFIG['model_vgg16_path']}")
    raise FileNotFoundError(f"Không tìm thấy file mô hình VGG16 tại {CONFIG['model_vgg16_path']}")
model_vgg16 = model_vgg16.to(device)
model_vgg16.eval()

# 3. Tải mô hình AlexNet
model_alexnet = alexnet(pretrained=False)
model_alexnet.classifier[6] = nn.Linear(4096, num_classes)
if os.path.exists(CONFIG['model_alexnet_path']):
    logger.info(f"Tải mô hình AlexNet từ {CONFIG['model_alexnet_path']}")
    print(f"Tải mô hình AlexNet từ {CONFIG['model_alexnet_path']}...")
    model_alexnet.load_state_dict(torch.load(CONFIG['model_alexnet_path'], map_location=device))
else:
    logger.error(f"Không tìm thấy file mô hình AlexNet tại {CONFIG['model_alexnet_path']}")
    raise FileNotFoundError(f"Không tìm thấy file mô hình AlexNet tại {CONFIG['model_alexnet_path']}")
model_alexnet = model_alexnet.to(device)
model_alexnet.eval()

# 4. Trích xuất đặc trưng từ VGG16 để huấn luyện KNN
def extract_features(model, loader):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            feats = model.classifier[5].forward(model.classifier[4].forward(
                model.classifier[3].forward(model.classifier[2].forward(
                    model.classifier[1].forward(model.classifier[0].forward(
                        model.features(inputs).view(inputs.size(0), -1)))))))
            features.append(feats.cpu().numpy())
            labels.append(targets.numpy())
    return np.concatenate(features), np.concatenate(labels)

# Huấn luyện hoặc tải KNN
model_knn_path = CONFIG['model_knn_path']
if os.path.exists(model_knn_path):
    logger.info(f"Tải mô hình KNN từ {model_knn_path}")
    print(f"Tải mô hình KNN từ {model_knn_path}...")
    with open(model_knn_path, 'rb') as f:
        model_knn = pickle.load(f)
else:
    logger.info("Trích xuất đặc trưng từ VGG16 để huấn luyện KNN...")
    print("Trích xuất đặc trưng từ VGG16 để huấn luyện KNN...")
    train_features, train_labels = extract_features(model_vgg16, train_loader)

    logger.info(f"Huấn luyện KNN với K={CONFIG['k']}...")
    print(f"Huấn luyện KNN với K={CONFIG['k']}...")
    model_knn = KNeighborsClassifier(n_neighbors=CONFIG['k'])
    model_knn.fit(train_features, train_labels)

    with open(model_knn_path, 'wb') as f:
        pickle.dump(model_knn, f)
    logger.info(f"Mô hình KNN đã được lưu vào {model_knn_path}")
    print(f"Mô hình KNN đã được lưu vào {model_knn_path}")

# 5. Đánh giá và dự đoán bằng Ensemble
def evaluate_and_predict_ensemble(model_vgg16, model_alexnet, model_knn, val_loader):
    correct = 0
    total = 0
    logger.info("Đánh giá mô hình Ensemble (VGG16 + AlexNet + KNN)...")
    print("Đánh giá mô hình Ensemble (VGG16 + AlexNet + KNN)...")
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs_vgg16 = model_vgg16(inputs)
            probs_vgg16 = torch.softmax(outputs_vgg16, dim=1)
            outputs_alexnet = model_alexnet(inputs)
            probs_alexnet = torch.softmax(outputs_alexnet, dim=1)
            feats = model_vgg16.classifier[5].forward(model_vgg16.classifier[4].forward(
                model_vgg16.classifier[3].forward(model_vgg16.classifier[2].forward(
                    model_vgg16.classifier[1].forward(model_vgg16.classifier[0].forward(
                        model_vgg16.features(inputs).view(inputs.size(0), -1)))))))
            knn_probs = torch.tensor(model_knn.predict_proba(feats.cpu().numpy()), dtype=torch.float32).to(device)
            outputs = (probs_vgg16 + probs_alexnet + knn_probs) / 3
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            probs = outputs[0]
            confidence = torch.max(probs).item() * 100
            flower_name = class_names[predicted[0]]
            log_msg = f"Dự đoán (Ensemble): {flower_name}, Độ tin cậy: {confidence:.2f}%"
            logger.info(log_msg)
            print(log_msg)
            break

    accuracy = 100 * correct / total
    log_msg = f"Độ chính xác Ensemble trên tập kiểm tra: {accuracy:.2f}%"
    logger.info(log_msg)
    print(log_msg)

evaluate_and_predict_ensemble(model_vgg16, model_alexnet, model_knn, val_loader)

# 6. Dự đoán một ảnh cụ thể bằng Ensemble
def predict_single_image_ensemble(image_path, model_vgg16, model_alexnet, model_knn, transform, class_names):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs_vgg16 = model_vgg16(image)
        probs_vgg16 = torch.softmax(outputs_vgg16, dim=1)
        outputs_alexnet = model_alexnet(image)
        probs_alexnet = torch.softmax(outputs_alexnet, dim=1)
        feats = model_vgg16.classifier[5].forward(model_vgg16.classifier[4].forward(
            model_vgg16.classifier[3].forward(model_vgg16.classifier[2].forward(
                model_vgg16.classifier[1].forward(model_vgg16.classifier[0].forward(
                    model_vgg16.features(image).view(image.size(0), -1)))))))
        knn_probs = torch.tensor(model_knn.predict_proba(feats.cpu().numpy()), dtype=torch.float32).to(device)
        outputs = (probs_vgg16 + probs_alexnet + knn_probs) / 3
        probs = outputs[0]
        confidence = torch.max(probs).item() * 100
        predicted_idx = torch.argmax(probs).item()
        flower_name = class_names[predicted_idx]
        log_msg = f"Dự đoán ảnh (Ensemble): {flower_name}, Độ tin cậy: {confidence:.2f}%"
        logger.info(log_msg)
        print(f"Dự đoán ảnh (Ensemble): {flower_name}, Độ tin cậy: {confidence:.2f}%")

logger.info("Dự đoán một ảnh cụ thể với Ensemble (VGG16 + AlexNet + KNN):")
print("Dự đoán một ảnh cụ thể với Ensemble (VGG16 + AlexNet + KNN):")
predict_single_image_ensemble(CONFIG['image_path'], model_vgg16, model_alexnet, model_knn, data_transforms['val'], class_names)

logger.info("Kết thúc chương trình")
print("Kết thúc chương trình")

file_handler.close()

INFO:root:Bắt đầu chương trình với Ensemble VGG16, AlexNet và KNN
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/

Device: cuda
Số loài hoa: 17


INFO:root:Tải mô hình VGG16 từ /content/drive/MyDrive/model/flower17/vgg16_flower_model.pth
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.r

Tải mô hình VGG16 từ /content/drive/MyDrive/model/flower17/vgg16_flower_model.pth...


INFO:root:Tải mô hình AlexNet từ /content/drive/MyDrive/model/flower17/alexnet_flower_model.pth
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_lo

...

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

Dự đoán (Ensemble): bluebell, Độ tin cậy: 98.38%
Độ chính xác Ensemble trên tập kiểm tra: 81.25%
Dự đoán một ảnh cụ thể với Ensemble (VGG16 + AlexNet + KNN):


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

Dự đoán ảnh (Ensemble): tulip, Độ tin cậy: 99.98%
Kết thúc chương trình


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 